<h1  align="center"><b> TRANSFORMAÇÃO E DECOMPOSIÇÃO </b></h1>

`Objetivo Geral:` Importar a série temporal tratada dos dados pluviométricos do município de São Paulo e realizar as transformações (em busca de estacionalidade e normalidade) e decomposição (em tendência, sazonalidade e resíduo)

`Dados:` Os dados foram tratados e extraídos na pasta [Tratamento de Dados](../[2]%20Tratamento%20de%20Dados%20-%20Projeto%20Chuva/) do Projeto Chuva.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

<h2 align="center"><b> Importação e Decomposição </b></h2>

In [4]:
série_chuva = pd.read_csv('../[2] Tratamento de Dados - Projeto Chuva/Série Tratada - Chuva Mensal.csv', sep = ';', index_col = 0)
série_chuva = pd.Series(série_chuva['Chuva Mensal (mm)'], index = série_chuva.index)

In [6]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposição = seasonal_decompose(série_chuva, model = 'additive', period = 12)

![decomposição](./Gráficos/Decomposição.png)

&emsp;&emsp; O seasonal_decompose é uma função do pacote statsmodels que permite decompor uma série temporal em suas componentes: tendência, sazonalidade e resíduo. A decomposição de uma série temporal em suas componentes é uma técnica útil para análise de séries temporais, pois ajuda a entender melhor o comportamento da série e identificar padrões.

&emsp;&emsp; A função calcula a média móvel da série temporal e subtrai (no caso do modelo aditivo) ou divide (no caso do modelo multiplicativo) a série original para obter a tendência. Em seguida, a função calcula a média das sazonalidades em cada período e subtrai ou divide a série para obter a componente sazonal. Por fim, a função calcula o resíduo retirando a tendência e a sazonalidade da série original.

<h2 align="center"><b> Normalidade </b></h2>

In [ ]:
import scipy.stats as stats
stats.probplot(série_chuva, dist = 'norm', plot = plt)
plt.title('Normal QQ Plot')
plt.show()

![Normal QQ Plot](./Gráficos/Normal%20QQ%20Plot.png)


In [ ]:
import seaborn as sns
sns.histplot(série_chuva, kde = True)
plt.title('Histograma')
plt.show()

![Normal Histograma](./Gráficos/Histograma.png)

&emsp;&emsp; Os gráficos acima mostram que a série provavelmente não apresenta distribuição normal. Para confirmar essa hipótese vamos realizar o teste de Shapiro-Wilk.

### `Teste Shapiro-Wilk`
&emsp;&emsp; O Teste de Shapiro-Wilk é uma ferramenta estatística utilizada para verificar se uma amostra de dados segue uma distribuição normal. O teste retorna um valor p, se esse valor for maior que o nível de significância (geralmente 0,05), dizemos que a distribuição é normal.

&emsp;&emsp; No caso da série temporal não ser normal, convém tentar realizar uma transformação para que ela se aproxime de uma distribuição normal, pois isso facilita a análise e a modelagem. As transformações mais comuns são a transformação logarítmica e a transformação por raiz cúbica.

In [9]:
def teste_shapiro(série):
    e, p = stats.shapiro(série)
    print(f'Estatística de Teste = {e}')
    print(f'p-valor = {p}')
    print(f'Resultado: {"É Normal" if (p > 0.05) else "Não é Normal"}')

teste_shapiro(série_chuva)

Estatística de Teste = 0.9242801666259766
p-valor = 2.1469944013830303e-14
Resultado: Não é Normal


`Transformação Logarítmica:` Não é possível realizar a transformação logarítmica, pois a série possui valores iguais a zero. Caso fosse possível, a transformação logarítmica seria realizada da seguinte forma: 

In [20]:
série_chuva_log = np.log(série_chuva)

`Transformação por Raiz Cúbica:` A transformação por raiz cúbica não vai resolver completamente o problema de normalidade, mas vai melhorar significativamente a distribuição dos dados. Podemos realizar a transformação por raiz cúbica da seguinte forma:

In [10]:
série_chuva_cub = série_chuva ** (1/3)
teste_shapiro(série_chuva_cub)

Estatística de Teste = 0.9843666553497314
p-valor = 7.994796033017337e-05
Resultado: Não é Normal


![Normal QQ Plot - Transformação por Raiz Cúbica](./Gráficos/Normal%20QQ%20Plot%20-%20Transformação%20por%20Raiz%20Cúbica.png)
![Normal Histograma - Transformação por Raiz Cúbica](./Gráficos/Histograma%20-%20Transformação%20por%20Raiz%20Cúbica.png)

<h2 align="center"><b> Estacionariedade </b></h2>

### `Teste KPSS (Kwiatkowski-Phillips-Schmidt-Shin)`
&emsp;&emsp; O teste KPSS é uma ferramenta estatística utilizada para avaliar a estacionariedade de uma série temporal. Uma série temporal é considerada estacionária quando sua média e variância são constantes ao longo do tempo. Se a estatística do teste for maior que o valor crítico (geralmente no 5%), dizemos que a série é não estacionária.

&emsp;&emsp; No caso da série temporal não ser estacionária, convém tentar realizar uma transformação para que ela se aproxime de uma série estacionária, pois isso facilita a análise e a modelagem. 

In [13]:
import statsmodels.tsa.stattools as stattools
def teste_KPSS(série):
    kpss = stattools.kpss(série)
    print(f'Estatística de Teste = {kpss[0]}')
    print(f'p-valor = {kpss[1]}')
    print(f'Valores Críticos:')
    for chave, valor in kpss[3].items():
        print(f'    {chave}: {valor}')
    print(f'Resultado: {"Série Estacionária" if (kpss[0] < kpss[3]["5%"]) else "Série Não Estacionária"}')

In [14]:
teste_KPSS(série_chuva_cub)

Estatística de Teste = 0.01665997605880993
p-valor = 0.1
Valores Críticos:
    10%: 0.347
    5%: 0.463
    2.5%: 0.574
    1%: 0.739
Resultado: Série Estacionária


C:\Users\Atual\AppData\Local\Temp\ipykernel_2516\1965257708.py:3: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss = stattools.kpss(série)


&emsp;&emsp; No caso a série já é estacionária, então não é necessário realizar nenhuma transformação. Caso a série não fosse estacionária, poderíamos realizar a transformação por diferenciação, que consiste em subtrair o valor atual pelo valor anterior. A transformação por diferenciação pode ser realizada da seguinte forma:

In [ ]:
série_chuva_cub_diff = série_chuva_cub.diff().dropna()
série_chuva_cub_diff

<h2 align="center"><b> Autocorrelação </b></h2>

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plot_acf(série_chuva_cub, lags = 20)
plt.title('Função de Autocorrelação')
plt.show()

plot_pacf(série_chuva_cub, lags = 20)
plt.title('Função de Autocorrelação Parcial')
plt.show()

![Função de Autocorrelação](./Gráficos/Função%20de%20Autocorrelação.png)
![Função de Autocorrelação Parcial](./Gráficos/Função%20de%20Autocorrelação%20Parcial.png)

<h2 align="center"><b> Finalização e Exportação </b></h2>
&emsp;&emsp; Após a realização das transformações a série está pronta para ser utilizada na modelagem. Como a série só tinha problemas de normalidade, vamos exportar a série transformada por raiz cúbica sem diferenciação (série_chuva_cub).

In [73]:
série_chuva_cub.to_csv('Série Transformada - Chuva Mensal.csv', sep = ';')